In [1]:
import tensorflow.compat.v1 as tf
import numpy as np
import matplotlib.pyplot as plt
import pickle

%matplotlib inline
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='latin1')
    return dict

In [4]:
data = unpickle('cifar-10-batches-py/data_batch_2')
input_data = data['data']
input_data = np.reshape(input_data, (10000,3,32,32)).transpose(0,2,3,1)
label = np.array(data['labels'])
label = np.eye(10)[label]
label.shape

(10000, 10)

In [5]:
input_data.shape

(10000, 32, 32, 3)

In [6]:
def init_weights(shape):
    W = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(W)

In [7]:
def init_bias(shape):
    return tf.constant(0.1, shape=shape)

In [8]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

In [9]:
def max_pool2by2(x):
    return tf.nn.max_pool(x, ksize=[1, 2,2, 1], strides=[1,2,2,1], padding='SAME')

In [10]:
def convolution_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x, W) + b)

In [11]:
def dense_layer(input_layer, size):
    W = init_weights([int(input_layer.get_shape()[1]), size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b

In [12]:
x = tf.placeholder(tf.float32, shape=[None, 32, 32, 3])

In [13]:
x_image = tf.reshape(x, shape=[-1, 32, 32, 3])

In [14]:
y_true = tf.placeholder(tf.float32, shape=[None, 10])

In [15]:
conv1 = convolution_layer(x, [5,5,3,32])
max_pool_conv1 = max_pool2by2(conv1)

In [16]:
max_pool_conv1

<tf.Tensor 'MaxPool:0' shape=(?, 16, 16, 32) dtype=float32>

In [17]:
conv2 = convolution_layer(max_pool_conv1, [5,5,32,64])
max_pool_conv2 = max_pool2by2(conv2)

In [18]:
max_pool_conv2

<tf.Tensor 'MaxPool_1:0' shape=(?, 8, 8, 64) dtype=float32>

In [19]:
dense_input_flat = tf.reshape(max_pool_conv2, [-1, 8*8*64])

In [20]:
dense1 = tf.nn.relu(dense_layer(dense_input_flat, 1024))

In [21]:
drop_out_prob = tf.placeholder(tf.float32)

drop_out_layer = tf.nn.dropout(dense1, keep_prob=drop_out_prob)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [22]:
y_pred = dense_layer(drop_out_layer, 10)

In [23]:
loss_func = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [24]:
optimizer = tf.train.AdamOptimizer()
train = optimizer.minimize(loss_func)

In [25]:
init = tf.global_variables_initializer()

In [26]:
def load_batch(X, y, batch_size):
    indexes = np.random.randint(len(X),size=batch_size)
    return X[indexes, :, :, :], y[indexes, :]

In [27]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(100):
        
        batch_x, batch_y = load_batch(input_data, label, 30)
        
        sess.run(train, feed_dict={x:batch_x, y_true:batch_y, drop_out_prob:0.5})
        
        print('epoch', epoch+1)
        
        matches = tf.equal(tf.arg_max(y_pred,1), tf.arg_max(y_true, 1))
        
        acc = tf.reduce_mean(tf.cast(matches, tf.float32))
        
        test_x, test_y = load_batch(input_data, label, 5)
        
        print('training acc: ', sess.run(acc, feed_dict={x:test_x, y_true:test_y, drop_out_prob:1}))

UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node Conv2D (defined at /home/kam/.virtualenvs/dl4cv/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1751) ]]

Original stack trace for 'Conv2D':
  File "/home/kam/anaconda3/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/kam/anaconda3/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/kam/.virtualenvs/dl4cv/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/kam/.virtualenvs/dl4cv/lib/python3.7/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/home/kam/.virtualenvs/dl4cv/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 563, in start
    self.io_loop.start()
  File "/home/kam/.virtuale